## Ring Homomorphism from Vector to Polynomial 

In [16]:
import galois

GF = galois.GF(79)  # this should be the curve order in real usages 

# let the vector v be (v_1, v_2, ...,v_3), assign x values (1, 2, ..., n) to vector elements and 
# interpolate to polynomial

v1 = GF([1, 0, 1])
v2 = GF([4, 5, 3])
x = GF([1, 2, 3])

p1 = galois.lagrange_poly(x, v1)
p2 = galois.lagrange_poly(x, v2)

# addition 
p3 = galois.lagrange_poly(x, v1 + v2)
p4 = p1 + p2

for i in range(1, 4):
    assert p4(i) == p3(i)

# multiplication
p3 = galois.lagrange_poly(x, v1 * v2)  # Hadamard product
p4 = p1 * p2

for i in range(1, 4):
    assert p4(i) == p3(i)

# scalar multiplication
p3 = galois.lagrange_poly(x, v1 * 3)
p4 = p1 * 3

for i in range(1, 4):
    assert p4(i) == p3(i)

## R1CS over Finite Field

In [17]:
from random import randint

L = [[0, 0, 1, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 1, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 1, 0]]

R = [[0, 0, 0, 1, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 1, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 1]]

O = [[0, 0, 0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 0, 0, 1],
     [0, 1, 0, 0, 0, 0, 0, 0]]

L = GF(L)
R = GF(R)
O = GF(O)

x = GF(randint(0, 78))
y = GF(randint(0, 78))
z = GF(randint(8, 78))
u = GF(randint(0, 78))

# circuit
v1 = x * y
v2 = z * u
out = x * y * z * u

# witness array
s = GF([1, out, x, y, z, u, v1, v2])

# Ls ⊙ Rs = Os
result = L.dot(s) * R.dot(s) == O.dot(s)

assert result.all(), "system contains an inequality"

## Polynomial Interpolation

In [18]:
import numpy as np

xs = GF([1, 2, 3])


def interpolate(vec):
    return galois.lagrange_poly(xs, vec)


U_polys = np.apply_along_axis(interpolate, 0, L)
V_polys = np.apply_along_axis(interpolate, 0, R)
W_polys = np.apply_along_axis(interpolate, 0, O)

print("U polys", U_polys)
print("V polys", V_polys)
print("W polys", W_polys)

U polys [Poly(0, GF(79)) Poly(0, GF(79)) Poly(40x^2 + 37x + 3, GF(79))
 Poly(0, GF(79)) Poly(78x^2 + 4x + 76, GF(79)) Poly(0, GF(79))
 Poly(40x^2 + 38x + 1, GF(79)) Poly(0, GF(79))]
V polys [Poly(0, GF(79)) Poly(0, GF(79)) Poly(0, GF(79))
 Poly(40x^2 + 37x + 3, GF(79)) Poly(0, GF(79))
 Poly(78x^2 + 4x + 76, GF(79)) Poly(0, GF(79))
 Poly(40x^2 + 38x + 1, GF(79))]
W polys [Poly(0, GF(79)) Poly(40x^2 + 38x + 1, GF(79)) Poly(0, GF(79))
 Poly(0, GF(79)) Poly(0, GF(79)) Poly(0, GF(79))
 Poly(40x^2 + 37x + 3, GF(79)) Poly(78x^2 + 4x + 76, GF(79))]


## Computing h(x)

In [30]:
from galois import Poly

# create the vanishing poly up to n
# (x-1)(x-2)(x-3)
t_poly = Poly([1, 78], GF) * Poly([1, 77], GF) * Poly([1, 76], GF)

# (U⋅a)(V⋅a) = (W⋅a) + h(x)⋅t(x)
# h(x) = ((U⋅a)(V⋅a)−(W⋅a))/t(x) 
LHS = U_polys.dot(s) * V_polys.dot(s)
RHS = W_polys.dot(s)
h_poly, remainder = divmod((LHS - RHS), t_poly)

print(h_poly)
assert remainder == 0, "remainder not 0"

# check (U⋅a)(V⋅a) = (W⋅a) + h(x)⋅t(x)
assert LHS == RHS + h_poly * t_poly, "system contains inequality"

49x + 40
